## Creating and visualizing simplified mapper graphs

The mapper graphs created with the *stress* and *tissue* lenses show some interesting patterns, where nodes consisting of certain types of samples are closely connected and clustered together. For example, with the *tissue* lens, we can observe clusters of nodes containing samples labeled *flower*, *seed*, and *fruit*, etc. As such, if we grouped these clusters of connected nodes together into a single super-node, we can simplify our visualization. That is what we do in this notebook.

### import packages
First thing, import all the necessary packages and methods. Most of these are commonly used python packages, that can be easily installed. We are assuming the mapper graph is already created and saved as `/data/saved_mapper_graphs/*.pkl` file under the project root directory. The node groupings are created manually, and a file mapping each node to a group label is available as `/data/node_group_mappings/*.csv` under the project root directory as well.

In addition, we will need to import a few custom functions defined in `helper_functions.py`. As the name suggests, it contains several utility functions required to perform tasks such as loading and scaling input data, or creating output files to be used by another application.

In [ ]:
# imports
import os
import numpy as np
import pandas as pd
import pickle

from scipy.spatial.distance import cdist

# Helper functions
from helper_functions import loaddata, get_color_dict
from helper_functions import create_grouped_pie_chart_json

### set paths, specify file names
Next thing we want to do is set the paths to the directories containing `code`, `data`, and `results`. We also specify input files. The data to be analyzed is stored in two csv files. `clean_metadata.csv` contains the metadata, `clean_RNAseq_OutlierRemoved.csv` contains the gene expression data. 

The metadata file contains one sample per row, identified by its *SRA*. There are 3172 samples, and for each sample we have 7 descriptive attributes. We are particularly interested in three of them: namely, its plant *family*, *tissue* type and *stress* type. There are 16 different plant families, 8 tissue types and 10 stress types (including *healthy*) represented in the data. The RNAseq file, as the name suggests contains the gene expression for each sample, across a 6335 orthogroups. 

The third file, named `colors.pkl`, is a pickle file (that's a python specific file format!) that contains a mapping between colors (RGB and hex codes) and the different classes of the factors (*family*, *stress* and *tissue* type). Kepler mapper outputs an `html` file visualized in a browser window, other types of plots and figures are created using `seaborn` and `matplotlib`. We have set the color mapping in a file to ensure consistency across these plotting tools used in the project.

In [ ]:
projdir = "../.."

datadir = projdir + "/data"
grouped_datadir = datadir + "/node_group_mappings"
resdir = projdir + "/results"
codedir = projdir + "/code"

factorfile = datadir + "/clean_metadata.csv"
colorfile = datadir + "/colors.pkl"
rnafile = datadir + "/raw_RNAseq_OutlierRemoved.csv"

### Loading data

The input data (metadata and gene expression) is stored in two separate files, but samples can be matched using the *SRA*. The function `loaddata` does exactly that. It loads both files into pandas dataframes and merges them using the *SRA* as the key. We perform an inner join, to keep only those *SRAs* for which we have metadata as well as gene expression. The `get_color_dict` is another utility function which looks for an existing `color.pkl` file and loads it. If the file is not present, the function will create it and return a dictionary containing the color mappings.

In [ ]:
factors = ["stress", "tissue", "family"]
df, orthos = loaddata(factorfile, rnafile, factors)
color_dict = get_color_dict(df, factors, colorfile)

### Set factor and level, load mapper graph

Here, we set the factor, and the corresponding factor level that was used to construct the lens function and the saved mapper graph that we now want to simplify. In the paper, we presented mapper graphs using following combinations:
- filterfactor: *stress*, filterlevel: *healthy*
- filterfactor: *tissue*, filterlevel: *root*
- filterfactor: *tissue*, filterlevel: *leaf*

These variables are also used to construct the names of files and directories, such as the file in which the mapper graph was saved and the file in which the node-group mapping is saved. In the following two code cells, we will also load the mapper graph and the node-group mapping.

In [ ]:
# Set filterfactor, filterlevel
filterfactor, filterlevel = ("stress", "healthy")

# Load the node-group mapping for the mapper graph to be simplified
groupf = f"/{filterfactor}_{filterlevel}_mapper_node_group_mappings.csv"
groupfile = grouped_datadir + groupf

gdata = pd.read_csv(groupfile)
gdf = gdata.groupby(["GroupIndex", "Group"]).size().reset_index()

In [ ]:
if filterfactor == "stress":
    cubes, overlap = (110, 75)
elif filterfactor == "tissue":
    if filterlevel == "root":
        cubes, overlap = (110, 90)
    elif filterlevel == "leaf":
        cubes, overlap = (140, 90)
    else:
        print("filterlevel must be root or leaf")
else:
    print("filterfactor must be stress or tissue")

mapf = f"/{filterfactor}_{filterlevel}_mapper_cubes_{cubes}_overlap_{overlap}"
mapperfile = datadir + "/saved_mapper_graphs/" + mapf + ".pkl"

with open(mapperfile, "rb") as fp:
    graph = pickle.load(fp)


### Compute sample-group mappings

The node-group mapping assigns each node of the mapper graph to a group. However, a node in the mapper graph consists of several individual samples. Some samples belong to more than one node of the mapper graph, forming the edges connecting those nodes. We use the node membership data from the mapper graph and the node-group mappings to map individual samples to the manually created groups for simplification. This mapping is needed to create the pie-chart visualization later.

We also compute a *centroid* for each group which can be used to assign each sample to a unique group if required in further analysis.

In [ ]:
# Compute cluster centroids for groups
centers = []
group_dict = dict()
for tup in gdf.itertuples():
    idx = tup.GroupIndex
    name = tup.Group
    clusters = gdata[gdata["GroupIndex"]==idx]["NodeID"].tolist()
    samples = []
    for c in clusters:
        samples += graph["nodes"][c]

    samples = list(set(samples))
    group_dict[idx] = {"name": name, "clusters": clusters, "samples": samples}
    center = np.mean(df.iloc[samples][orthos].to_numpy(), axis=0, keepdims=True)
    centers.append(center)

centers = np.asarray(centers).squeeze(axis=1)
print(centers.shape)

We can map each sample to a unique group, based on the distance between the sample gene expression and the closest group *centroid*.

In [ ]:
# Assign each sample to unique group based on closest centroid
data = df[orthos].to_numpy()
dmat = cdist(data, centers, metric="correlation")
grpidx = dmat.argmin(axis=1)
df.insert(6, "group", grpidx+1)

Uncomment the following cell block if you wish to save the sample-group mappings. The mapping where a sample is allowed to be mapped to more than one group is saved as a `.pkl` file. The unique sample-group mappings are saved in a `.csv` file along with gene expression and other metadata corresponding to the samples. This file is large: close to 350 MB in size.

In [ ]:
# # Uncomment to save sample group mappings to files
# fname1 = f"/{filterfactor}_{filterlevel}_mapper_group_sample_mapping.pkl"
# with open(grouped_datadir + fname1, "wb") as fp:
#     pickle.dump(group_dict, fp, protocol=pickle.HIGHEST_PROTOCOL)

# fname2 = f"/{filterfactor}_{filterlevel}_mapper_group_assigned_dataframe.csv"
# df.to_csv(grouped_datadir + fname2, index=False)

### Simplified mapper with pie-chart nodes

In this next part, we want to visualize the nodes of the simplified mapper graph in the form of pie-charts. We will replace each node by a pie-chart that shows the distribution of group memebrs with respect to a given factor. The function `create_grouped_pie_chart_json` computes the corresponding pie-charts for each group of the simplified mapper graph and stores the information in a json file. This json file is then pulled by the javascript code to create the visualization.

In [ ]:
json_dir = codedir + "/kepler_mapper_pie_nodes/json_data"
_ = create_grouped_pie_chart_json(graph, df, gdata, factors,
                                   filterfactor, filterlevel,
                                   color_dict, json_dir)

### Visualizing the simplified mapper with pie-chart nodes

After the json files are created and saved, open a terminal and go to the `code/kepler_mapper_pie_nodes` directory under the project root. Then run `python -m http.server` and once the server is running, open `http://0.0.0.8000` or `localhost:8000` in your browser. This will open up a directory listing with several html files. The filesnames follow the following pattern: 
- `grouped_<factor>_mapper_<filterfactor>_<filterlevel>_lens.html`

The *grouped_* prefix indicates that this is the simplified mapper graph. The lens for the mapper graph was computed using *filterfactor*, and *filterlevel*, and the pie-charts show the distribution of *factor* for each node. For example, the file `grouped_family_mapper_tissue_leaf_lens.html` contains simplified mapper graphs computed using a *tissue* lens measuring deviation from the idealized *leaf* expression and the pie charts show the distribution of plant *family* across node members.